In [1]:
import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns
sns.reset_orig()

from scipy.stats import bernoulli
from scipy.special import loggamma

from sklearn.preprocessing import OneHotEncoder

import tqdm

from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr

In [2]:
import os
import pickle

In [3]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [4]:
indir = './data'
outdir = './output'
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [5]:
blockmodels = importr("blockmodels")

In [6]:
base = importr("base")

In [7]:
dollar = base.__dict__["$"]

In [8]:
from functools import lru_cache

@lru_cache(maxsize = 10000)
def normterm_discrete(n, k):
    if n == 1:
        return np.log(k)
    if k == 1:
        return 1.0
    elif k == 2:
        return np.sum(sorted([ np.exp(loggamma(n+1) - loggamma(t+1) - loggamma(n-t+1) + 
                               t*(np.log(t) - np.log(n)) + (n-t)*(np.log(n-t) - np.log(n))
                        )
                        for t in range(1, n)]))
    else:
        return normterm_discrete(n, k-1) + n/(k-2) * normterm_discrete(n, k-2)

In [9]:
def calc_dnml(X, Z1, Z2, K=3, L=3):    
    N = X.shape[0]
    
    codelen_x_z = 0.0
    codelen_z = 0.0

    for k in range(K):
        for l in range(L):
            n_pos = np.sum(X[Z1 == k, :][:, Z2 == l] == 1)
            n_neg = np.sum(X[Z1 == k, :][:, Z2 == l] == 0)
            n_all = n_pos + n_neg

            if n_all >=2:
                codelen_x_z += n_all * np.log(n_all)
                codelen_x_z += np.log(normterm_discrete(n_all, 2))
                #codelen += n_all * np.log(n_all)
                #codelen += np.log(normterm_discrete(n_all, 2))
            
            if n_pos >=2:
                #codelen -= n_pos * np.log(n_pos)
                codelen_x_z -= n_pos * np.log(n_pos)
            if n_neg >=2:
                #codelen -= n_neg * np.log(n_neg)
                codelen_x_z -= n_neg * np.log(n_neg)
            
            #codelen += codelen_x_z

        n_k = np.sum(Z1 == k)
        if n_k >= 1:
            #codelen += n_k * (np.log(N) - np.log(n_k))
            codelen_z += n_k * (np.log(N) - np.log(n_k))

    #codelen += np.log(normterm_discrete(N, K))
    codelen_z += np.log(normterm_discrete(N, K))
    
    codelen = codelen_x_z + codelen_z

    #return codelen
    return codelen, codelen_x_z, codelen_z

In [10]:
def calc_lsc(X, Z1, Z2, K=3, L=3):    
    codelen = 0.0
    N = X.shape[0]

    for k in range(K):
        for l in range(L):
            n_pos = np.sum(X[Z1 == k, :][:, Z2 == l] == 1)
            n_neg = np.sum(X[Z1 == k, :][:, Z2 == l] == 0)
            n_all = n_pos + n_neg

            if n_all >=2:
                codelen += n_all * np.log(n_all)
            
            if n_pos >=2:
                codelen -= n_pos * np.log(n_pos)
            if n_neg >=2:
                codelen -= n_neg * np.log(n_neg)

        n_k = np.sum(Z1 == k)
        if n_k >= 1:
            codelen += n_k * (np.log(N) - np.log(n_k))

        codelen += (k + (k+1)*(k+2))/2 * np.log(N/(2.0*np.pi)) -(k+1)/2 * np.log(2.0) + \
           (k+1) * loggamma((k+3)/2) - loggamma((k+1)*(k+3)/2) + (k+1)*(k+2)/2 * np.log(np.pi)

    return codelen

In [11]:
with open(os.path.join(indir, 'X_abrupt.pkl'), 'rb') as f:
    X_all = pickle.load(f)
with open(os.path.join(indir, 'Z_abrupt.pkl'), 'rb') as f:
    Z_all = pickle.load(f)

In [12]:
X_all.shape

(10, 80, 100, 100)

In [13]:
cluster_range_max = 6

In [14]:
EPS = np.finfo(np.float).eps

T = X_all.shape[1]

pi1 = None
pi2 = None
a0 = 1.0
b0 = 1.0
ratio = 0.02

pi_all = []
theta_all = []
z_all = []

for trial in range(10):
    print('trial =', trial)
    pi1 = None
    pi2 = None
    theta = None
        
    pi_list_trial = []
    theta_list_trial = []
    z_list_trial = []
    
    for t in tqdm.tqdm(range(T)):
        seed = trial*T + t
        
        X = X_all[trial, t, :, :]
            
        numpy2ri.activate()
        sbm = blockmodels.BM_bernoulli(membership_type="SBM", adj=np.array(X),
                                           verbosity=0,
                                           exploration_factor=1.5,
                                           explore_min=cluster_range_max,
                                           explore_max=cluster_range_max)

        estimate = dollar(sbm, "estimate")
        estimate()

        theta_list = []
        pi_list = []
        z_posterior_list = []
        for k in range(cluster_range_max):
            n_clusters = k + 1
            theta = np.array(dollar(dollar(sbm, "model_parameters")[k], "pi"))
            z_posterior = np.array(dollar(dollar(sbm, "memberships")[k], "Z"))
            pi = np.sum(z_posterior, axis=0) + 10 * EPS
            pi /= np.sum(pi)

            theta_list.append(theta)
            z_posterior_list.append(z_posterior)
            pi_list.append(pi)

        numpy2ri.deactivate()
        
        pi_list_trial.append(pi_list)
        theta_list_trial.append(theta_list)
        z_list_trial.append(z_posterior_list)
        
    pi_all.append(pi_list_trial)
    theta_all.append(theta_list_trial)
    z_all.append(z_list_trial)

    with open(os.path.join(outdir, 'pi_abrupt.pkl'), 'wb') as f:
        pickle.dump(pi_all, f)
    with open(os.path.join(outdir, 'theta_abrupt.pkl'), 'wb') as f:
        pickle.dump(theta_all, f)
    with open(os.path.join(outdir, 'z_abrupt.pkl'), 'wb') as f:
        pickle.dump(z_all, f)

  0%|          | 0/80 [00:00<?, ?it/s]

trial = 0



  1%|▏         | 1/80 [00:01<01:26,  1.10s/it]

  2%|▎         | 2/80 [00:01<01:19,  1.02s/it]

  4%|▍         | 3/80 [00:02<01:14,  1.03it/s]

...

 95%|█████████▌| 76/80 [01:24<00:03,  1.05it/s]

 96%|█████████▋| 77/80 [01:25<00:02,  1.05it/s]

 98%|█████████▊| 78/80 [01:26<00:01,  1.04it/s]

 99%|█████████▉| 79/80 [01:27<00:00,  1.04it/s]

100%|██████████| 80/80 [01:28<00:00,  1.01it/s]
